### Build and test a Naive Bayes classifier.

In [94]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from sklearn.datasets import load_iris
from sklearn.naive_bayes import BernoulliNB

In [95]:
# Load the data, which is included in sklearn.
iris = load_iris()
print ('Iris target names:', iris.target_names)
print ('Iris feature names:', iris.feature_names)
X, Y = iris.data, iris.target

# Shuffle the data, but make sure that the features and accompanying labels stay in sync.
np.random.seed(0)
shuffle = np.random.permutation(np.arange(X.shape[0]))
X, Y = X[shuffle], Y[shuffle]

# Split into train and test.
train_data, train_labels = X[:100], Y[:100]
test_data, test_labels = X[100:], Y[100:]

Iris target names: ['setosa' 'versicolor' 'virginica']
Iris feature names: ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


The iris feature values are real valued -- measurements in centimeters. Let's look at histograms of each feature.

To make things simple, let's binarize these feature values. That is, we'll treat each measurement as either "short" or "long". I'm just going to choose a threshold for each feature.

In [96]:
# Define a function that applies a threshold to turn real valued iris features into 0/1 features.
# 0 will mean "short" and 1 will mean "long".
def binarize_iris(data, thresholds=[6.0, 3.0, 2.5, 1.0]):
    # Initialize a new feature array with the same shape as the original data.
    binarized_data = np.zeros(data.shape)

    # Apply a threshold  to each feature.
    for feature in range(data.shape[1]):
        binarized_data[:,feature] = data[:,feature] > thresholds[feature]
    return binarized_data

# Create new binarized training and test data
binarized_train_data = binarize_iris(train_data)
binarized_test_data = binarize_iris(test_data)

print (train_data[:10, ])
print (binarized_train_data[:10, ])

[[5.8 2.8 5.1 2.4]
 [6.  2.2 4.  1. ]
 [5.5 4.2 1.4 0.2]
 [7.3 2.9 6.3 1.8]
 [5.  3.4 1.5 0.2]
 [6.3 3.3 6.  2.5]
 [5.  3.5 1.3 0.3]
 [6.7 3.1 4.7 1.5]
 [6.8 2.8 4.8 1.4]
 [6.1 2.8 4.  1.3]]
[[0. 0. 1. 1.]
 [0. 0. 1. 0.]
 [0. 1. 0. 0.]
 [1. 0. 1. 1.]
 [0. 1. 0. 0.]
 [1. 1. 1. 1.]
 [0. 1. 0. 0.]
 [1. 1. 1. 1.]
 [1. 0. 1. 1.]
 [1. 0. 1. 1.]]


Recall that Naive Bayes assumes conditional independence of features. With $Y$ the set of labels and $X$ the set of features ($y$ is a specific label and $x$ is a specific feature), Naive Bayes gives the probability of a label $y$ given input features $X$ as:

$ \displaystyle P(y|X) \approx 
  \frac{P(y) \prod _{x\in X} P(x|y)}{P(X)}
  =  \frac { P(y) \prod_{x \in X} P(x|y) }
        { \sum_{y \in Y} P(y) \prod_{x \in X} P(x|y) }
$

Let's estimate some of these probabilities using maximum likelihood, which is just a matter of counting and normalizing. We'll start with the prior probability of the label $P(y)$.

In [97]:
# Initialize counters for all labels to zero.
label_counts = [0 for i in iris.target_names]
# Iterate over labels in the training data and update counts.
for label in train_labels:
    label_counts[label] += 1
    
print(label_counts)
# Normalize counts to get a probability distribution.
total = sum(label_counts)
label_probs = [1.0 * count / total for count in label_counts]
for (prob, name) in zip(label_probs, iris.target_names):
    print ('%15s : %.2f%%' %(name, prob))

[31, 33, 36]
         setosa : 0.31%
     versicolor : 0.33%
      virginica : 0.36%


Next, let's estimate $P(X|Y)$, that is the probability of each feature given each label. Remember that we can get the conditional probability from the joint distribution:

$\displaystyle P(X|Y) = \frac{ P(X,Y) } { P(Y) } \approx \frac{ \textrm{Count}(X,Y) } { \textrm{Count}(Y) }$

Let's think carefully about the size of the count matrix we need to collect. There are 3 labels $y_0$, $y_1$, and $y_2$ and 4 features $x_0$, $x_1$, $x_2$, and $x_3$. Each feature has 2 possible values, 0 or 1. So there are actually $4 \times 2 \times 3=24$ probabilities we need to estimate: 

$P(x_0=0, Y=y_0)$

$P(x_0=1, Y=y_0)$

$P(x_1=0, Y=y_0)$

$P(x_1=1, Y=y_0)$

...

However, we already estimated (above) the probability of each label. And, we know that each feature value is either 0 or 1. So, for example,

$P(x_0=0, Y=\textrm{setosa}) + P(x_0=1, Y=\textrm{setosa}) = P(Y=\textrm{setosa}) \approx 0.31$.

As a result, we can just estimate probabilities for one of the feature values, say, $x_i = 0$. This requires a $4 \times 3$ matrix.

In [98]:
# Initialize a matrix for joint counts of feature=0 and label.
feature0_and_label_counts = np.zeros([len(iris.feature_names), len(iris.target_names)])

# Just to check our work, let's also keep track of joint counts of feature=1 and label.
feature1_and_label_counts = np.zeros([len(iris.feature_names), len(iris.target_names)])

for i in range(binarized_train_data.shape[0]):
    for feature_index, feature_value in enumerate(binarized_train_data[i]):
        feature0_and_label_counts[feature_index][train_labels[i]] += (feature_value == 0)
        feature1_and_label_counts[feature_index][train_labels[i]] += (feature_value == 1)
# Let's look at the counts.
print ('Feature = 0 and label:\n', feature0_and_label_counts)
print ('\nFeature = 1 and label:\n', feature1_and_label_counts)

# As a sanity check, what should the total sum of all counts be?
# We have 100 training examples, each with 4 features. So we should have counted 400 things.
total_sum = feature0_and_label_counts.sum() + feature1_and_label_counts.sum()
print ('\nTotal count:', total_sum)
# As another sanity check, the label probabilities should be equal to the normalized feature counts for each label.
print ('Label probabilities:', (feature0_and_label_counts.sum(0) + feature1_and_label_counts.sum(0)) / total_sum)


Feature = 0 and label:
 [[31. 20.  7.]
 [ 6. 27. 25.]
 [31.  0.  0.]
 [31.  3.  0.]]

Feature = 1 and label:
 [[ 0. 13. 29.]
 [25.  6. 11.]
 [ 0. 33. 36.]
 [ 0. 30. 36.]]

Total count: 400.0
Label probabilities: [0.31 0.33 0.36]


We still need to normalize the joint counts to get probabilities.

In [99]:
# Initialize new matrices to hold conditional probabilities.
feature0_given_label = np.zeros(feature0_and_label_counts.shape)
feature1_given_label = np.zeros(feature1_and_label_counts.shape)

# P(feature|label) = P(feature, label) / P(label) =~ count(feature, label) / count(label).
# Note that we could do this normalization more efficiently with array operations, but for the sake of clarity,
# let's iterate over each label and each feature.
for label in range(feature0_and_label_counts.shape[1]):
    for feature in range(feature0_and_label_counts.shape[0]):
        feature0_given_label[feature][label]= feature0_and_label_counts[feature][label]/label_counts[label]
        feature1_given_label[feature][label]= feature1_and_label_counts[feature][label]/label_counts[label]
                
# Here's our estimated conditional probability table.
print ('Estimated values of P(feature=0|label):\n', feature0_given_label)

print ('Estimated values of P(feature=1|label):\n', feature1_given_label)

# As a sanity check, which probabilities should sum to 1?
print ('\nCheck that P(feature=0|label) + P(feature=1|label) = 1\n',feature0_given_label + feature1_given_label)

Estimated values of P(feature=0|label):
 [[1.         0.60606061 0.19444444]
 [0.19354839 0.81818182 0.69444444]
 [1.         0.         0.        ]
 [1.         0.09090909 0.        ]]
Estimated values of P(feature=1|label):
 [[0.         0.39393939 0.80555556]
 [0.80645161 0.18181818 0.30555556]
 [0.         1.         1.        ]
 [0.         0.90909091 1.        ]]

Check that P(feature=0|label) + P(feature=1|label) = 1
 [[1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]
 [1. 1. 1.]]


Now that we have all the pieces, let's try making a prediction for the first test example. It looks like this is a setosa (label 0) example with all small measurements -- all the feature values are 0.

We start by assuming the prior distribution, which has a slight preference for virginica, followed by versicolor. Of course, these estimates come from our training data, which might not be a representative sample. In practice, we may prefer to use a uniform prior.

In [100]:
# What does the feature vector look like? And what's the true label?
index = 0
print ('Feature vector:', binarized_test_data[index])
print ('Label:', test_labels[index])

Feature vector: [0. 0. 0. 0.]
Label: 0


We calculate for each label the probability that the test example with its specific features has this label.

\begin{align}
P(Y=s|X) &\approx P(x_0=0|Y=s)P(x_1=0|Y=s)P(x_2=0|Y=s)P(x_3=0|Y=s)P(Y=s)\\
P(Y=ve|X) &\approx P(x_0=0|Y=ve)P(x_1=0|Y=ve)P(x_2=0|Y=ve)P(x_3=0|Y=ve)P(Y=ve)\\
P(Y=vi|X) &\approx P(x_0=0|Y=vi)P(x_1=0|Y=vi)P(x_2=0|Y=vi)P(x_3=0|Y=vi)P(Y=vi)\\
\end{align}

In [101]:
# Let's include the first feature. We use feature1_given_label since the feature value is 1.

# Multiply by each conditional feature probability.
predictions = label_probs.copy()
for item in binarized_train_data:
    for (index, value) in enumerate(item):
        feature_probs = feature1_given_label[index]
        if not value: feature_probs = 1 - feature_probs
        predictions *= feature_probs
        
print('Before normalizing:', predictions)
# We could wait until we've multiplied by all the feature probabilities, 
# but there's no harm in normalizing after each update.
predictions /= predictions.sum()
print ('After normalizing:', predictions)

Before normalizing: [0. 0. 0.]
After normalizing: [nan nan nan]


C:\Users\dbwjd\AppData\Local\Temp\ipykernel_28112\76526421.py:14: RuntimeWarning: invalid value encountered in divide
  predictions /= predictions.sum()


This also predicts label 0.

What happened?

Well, it looks like Naive Bayes came up with the right answer. But it seems overconfident!

Let's look again at our conditional probability estimates for the features. Notice that there are a bunch of zero probabilities. This is bad because as soon as we multiply anything by zero, we're guaranteed that our final estimate will be zero. This is an overly harsh penalty for an observation that simply never occurred in our training data. Surely there's some possibility, even if very small, that there could exist a setosa with a long sepal.

This is where smoothing comes in. The maximum likelihood estimate is only optimal in the case where we have infinite training data. When we have less than that, we need to temper maximum likelihood by reserving some small probability for unseen events. The simplest way to do this is with Laplace smoothing -- rather than starting with a count of 0 for each joint (feature, label) observation, we start with a count of $\alpha$.

$\displaystyle P(X|Y) = \frac{ P(X,Y) } { P(Y) } \approx \frac{ \textrm{Count}(X,Y) + \alpha } { \textrm{Count}(Y)  + \alpha|X|} \stackrel{\text{here}}{=} \frac{ \textrm{Count}(X,Y) + \alpha } { \textrm{Count}(Y)  +  \alpha 2 }$

Let's package training and inference into a class, modeled after sklearn's BernoulliNB.

In [102]:
class NaiveBayes:
    # Initialize an instance of the class.
    def __init__(self, alpha=0.0):
        self.alpha = alpha     # additive (Laplace) smoothing parameter
        self.priors = None     # estimated by fit()
        self.probs = None      # estimated by fit()
        self.num_labels = 0    # set by fit()
        self.num_features = 0  # set by fit()
        
    def fit(self, train_data, train_labels):
        # Store number of labels, number of features, and number training examples.
        self.num_labels = len(np.unique(train_labels))
        self.num_features = train_data.shape[1]
        self.num_examples = train_data.shape[0]
        # Initialize an array of label counts. Each label gets a smoothed count of 2*alpha because
        # each feature value (0 and 1) gets an extra count of alpha.
        label_counts = np.ones(self.num_labels) * self.alpha * 2
        
        # Initialize an array of (feature=1, label) counts to alpha.
        feature1_and_label_counts = np.ones([self.num_features, self.num_labels]) * self.alpha
        
        # Count features with value == 1.
        for i in range(self.num_examples):
            #YOUR CODE
            thresholds = [6.0, 3.0, 2.5, 1.0]
            for x in range(len(thresholds)):
                if train_data[i][x] > thresholds[x]:
                    feature1_and_label_counts[x][train_labels[i]] += 1 


        # Normalize to get probabilities P(feature=1|label).
        #YOUR CODE   
        label_counts = np.sum(feature1_and_label_counts, axis=0) + label_counts
        self.probs = np.zeros(feature1_and_label_counts.shape)
        for label in range(feature1_and_label_counts.shape[1]):
            for feature in range(feature1_and_label_counts.shape[0]):
                self.probs[feature][label]= feature1_and_label_counts[feature][label]/label_counts[label]
        # Normalize label counts to get prior probabilities P(label).
        #self.priors = #YOUR CODE
        priors = (train_labels[:, np.newaxis] == np.array([0, 1, 2])[np.newaxis, :])
        self.priors = np.sum(priors, axis=0) /  len(train_labels)       
        

    # Make predictions for each test example and return results.
    def predict(self, test_data):
        results = []
        for item in test_data:
            results.append(self._predict_item(item))
        return np.array(results)
    
    # Private function for making a single prediction.
    def _predict_item(self, item):
        # Make a copy of the prior probabilities.
        predictions = self.priors.copy()
        
        # Multiply by each conditional feature probability.
        for (index, value) in enumerate(item):
            thresholds = [6.0, 3.0, 2.5, 1.0]
            feature_probs = self.probs[index]
            if not (value > thresholds[index]): feature_probs = 1 - feature_probs
            predictions *= feature_probs
 
        # Normalize and return the label that gives the largest probability.
        ###YOUR CODE
        predictions /= predictions.sum()
        return predictions.argmax()

Now we can compare our implementation with the sklearn implementation. Do the predictions agree? What about the estimated parameters? Try changing alpha from 0 to 1.

Note: I think there might be a bug in the sklearn code. What do you think?

In [103]:
alpha = 1
nb = NaiveBayes(alpha=alpha)
nb.fit(train_data, train_labels)

# Compute accuracy on the test data.
preds = nb.predict(test_data)
correct, total = 0, 0
for pred, label in zip(preds, test_labels):
    if pred == label: correct += 1
    total += 1
print ('With alpha = %.2f' %alpha)
print ('[OUR implementation] total: %3d  correct: %3d  accuracy: %3.2f' %(total, correct, 1.0*correct/total))

# Compare to sklearn's implementation.
clf = BernoulliNB(alpha=alpha, fit_prior=False)
clf.fit(binarized_train_data, train_labels)
print ('sklearn accuracy: %3.2f' %clf.score(binarized_test_data, test_labels))

print ('\nOur feature probabilities\n', nb.probs)
print ('\nsklearn feature probabilities\n', np.exp(clf.feature_log_prob_).T)

print ('\nOur prior probabilities\n', nb.priors)
print ('\nsklearn prior probabilities\n', np.exp(clf.class_log_prior_))


100
With alpha = 1.00
[OUR implementation] total:  50  correct:  39  accuracy: 0.78
sklearn accuracy: 0.82

Our feature probabilities
 [[0.03225806 0.15909091 0.25423729]
 [0.83870968 0.07954545 0.10169492]
 [0.03225806 0.38636364 0.31355932]
 [0.03225806 0.35227273 0.31355932]]

sklearn feature probabilities
 [[0.03030303 0.4        0.78947368]
 [0.78787879 0.2        0.31578947]
 [0.03030303 0.97142857 0.97368421]
 [0.03030303 0.88571429 0.97368421]]

Our prior probabilities
 [0.31 0.33 0.36]

sklearn prior probabilities
 [0.33333333 0.33333333 0.33333333]
2.8958074456980962
6.388562314878105
